In [50]:
# Задание на повторение материала предыдущего семестра

In [51]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [52]:
# Генерируем уникальный seed
my_code = "Nosova"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [53]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [54]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [55]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [56]:
example_data.shape

(2315, 7)

In [57]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [58]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [59]:
train.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
2044,2014,Astrakhan Oblast,2.3,14.9,12.6,69.58,66.70
2368,2017,Udmurt Republic,-0.2,11.8,12.0,80.39,65.70
2249,2016,Moscow,1.8,11.8,10.0,68.44,98.80
2001,2013,Omsk Oblast,1.3,14.7,13.4,64.69,71.80
1173,2003,Republic of Tatarstan,-3.6,10.2,13.8,65.67,73.88


In [60]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [61]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [62]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [63]:
sc_val

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.62963,0.964194,0.912664,0.07619,0.552925,0.13816,Chechen Republic
1,0.111111,0.616368,0.615721,0.395238,0.777523,0.312782,Tuva Republic
2,0.777778,0.317136,0.231441,0.533333,0.430111,0.738796,Samara Oblast
3,0.962963,0.634271,0.414847,0.142857,0.347411,0.896579,Khanty–Mansi Autonomous Okrug – Yugra
4,0.148148,0.409207,0.305677,0.442857,0.764269,0.605277,Republic of North Ossetia-Alania
...,...,...,...,...,...,...,...
458,0.296296,0.227621,0.087336,0.542857,0.689875,0.612702,Kirov Oblast
459,0.0,0.585678,0.436681,0.257143,0.54939,0.580615,Amur Oblast
460,0.740741,0.40665,0.340611,0.485714,0.400424,0.473614,Republic of Bashkortostan
461,0.666667,0.547315,0.436681,0.32381,0.257466,0.525325,Sakha (Yakutia) Republic


In [64]:
# Вспоминаем алгоритмы решения задачи регрессии
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
r_models.append(SVR(kernel='linear'))
r_models.append(SVR(kernel='poly'))
r_models.append(SVR(kernel='rbf'))
r_models.append(SVR(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'mse', 'friedman_mse', 'mae', 'poisson'
r_models.append(DecisionTreeRegressor(criterion='squared_error'))
r_models.append(DecisionTreeRegressor(criterion='friedman_mse'))
r_models.append(DecisionTreeRegressor(criterion='absolute_error'))

In [65]:
r_models

[LinearRegression(),
 Lasso(),
 Ridge(),
 ElasticNet(),
 KNeighborsRegressor(),
 KNeighborsRegressor(n_neighbors=10),
 KNeighborsRegressor(n_neighbors=15),
 SVR(kernel='linear'),
 SVR(kernel='poly'),
 SVR(),
 SVR(kernel='sigmoid'),
 DecisionTreeRegressor(),
 DecisionTreeRegressor(criterion='friedman_mse'),
 DecisionTreeRegressor(criterion='absolute_error')]

In [66]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [67]:
x_labels

['year', 'npg', 'birth_rate', 'death_rate', 'gdw']

In [68]:
y_labels

['urbanization']

In [69]:
# Обучаем модели
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

LinearRegression()
Lasso()
Ridge()
ElasticNet()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [70]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

LinearRegression() 	 0.01834433204835998
Lasso() 	 0.0286963434757694
Ridge() 	 0.018392164014424426
ElasticNet() 	 0.0286963434757694
KNeighborsRegressor() 	 0.010224574988776347
KNeighborsRegressor(n_neighbors=10) 	 0.011371854612736237
KNeighborsRegressor(n_neighbors=15) 	 0.012250175459242884
SVR(kernel='linear') 	 0.01855678205468759
SVR(kernel='poly') 	 0.012122079525248231
SVR() 	 0.011212067763147662
SVR(kernel='sigmoid') 	 35.31678873831897
DecisionTreeRegressor() 	 0.02118213729752154
DecisionTreeRegressor(criterion='friedman_mse') 	 0.021577358573875247
DecisionTreeRegressor(criterion='absolute_error') 	 0.017938636223346183


In [71]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [72]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.009967530769133145


In [73]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l1', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='linear'))
c_models.append(SVC(kernel='poly'))
c_models.append(SVC(kernel='rbf'))
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='random'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='random'))

In [74]:
c_models

[LogisticRegression(max_iter=500, penalty='none', solver='saga'),
 LogisticRegression(max_iter=500, penalty='l1', solver='saga'),
 LogisticRegression(solver='saga'),
 LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'),
 LogisticRegression(),
 MultinomialNB(alpha=0.0),
 MultinomialNB(alpha=0.5),
 MultinomialNB(),
 KNeighborsClassifier(),
 KNeighborsClassifier(n_neighbors=10),
 KNeighborsClassifier(n_neighbors=15),
 SVC(kernel='linear'),
 SVC(kernel='poly'),
 SVC(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random'),
 DecisionTreeClassifier(criterion='entropy'),
 DecisionTreeClassifier(criterion='entropy', splitter='random')]

In [75]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [76]:
x_train

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.888889,0.450128,0.379913,0.447619,0.54992,0.558473
1,1.0,0.386189,0.244541,0.419048,0.740944,0.545213
2,0.962963,0.43734,0.244541,0.32381,0.529776,0.984089
3,0.851852,0.424552,0.371179,0.485714,0.463509,0.626094
4,0.481481,0.299233,0.174672,0.504762,0.480827,0.653673
...,...,...,...,...,...,...
1384,0.333333,0.2711,0.091703,0.466667,0.566885,0.576637
1385,0.888889,0.432225,0.222707,0.309524,0.442304,0.985415
1386,0.37037,0.199488,0.078603,0.585714,0.661954,0.536728
1387,0.185185,0.309463,0.087336,0.390476,0.271956,0.89764


In [77]:
# Обучаем модели
for model in c_models:
    print(model)
    model.fit(x_train, y_train)

LogisticRegression(max_iter=500, penalty='none', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(max_iter=500, penalty='l1', solver='saga')
LogisticRegression(solver='saga')
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')
LogisticRegression()
MultinomialNB(alpha=0.0)
MultinomialNB(alpha=0.5)


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


MultinomialNB()
KNeighborsClassifier()
KNeighborsClassifier(n_neighbors=10)
KNeighborsClassifier(n_neighbors=15)
SVC(kernel='linear')
SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')
DecisionTreeClassifier(criterion='entropy')
DecisionTreeClassifier(criterion='entropy', splitter='random')


In [78]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

LogisticRegression(max_iter=500, penalty='none', solver='saga') 	 0.35821519708473154
LogisticRegression(max_iter=500, penalty='l1', solver='saga') 	 0.1415672380142219
LogisticRegression(solver='saga') 	 0.05524557312238927
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga') 	 0.07096528562562224
LogisticRegression() 	 0.05524557312238927
MultinomialNB(alpha=0.0) 	 0.007418674234850861
MultinomialNB(alpha=0.5) 	 0.008879417410977178
MultinomialNB() 	 0.00883308909608604
KNeighborsClassifier() 	 0.385398867586144
KNeighborsClassifier(n_neighbors=10) 	 0.2887300556674809
KNeighborsClassifier(n_neighbors=15) 	 0.23484833868921298
SVC(kernel='linear') 	 0.07800974820366825
SVC(kernel='poly') 	 0.6487069614222587
SVC() 	 0.19462848521964815
SVC(kernel='sigmoid') 	 3.715831765716807e-05
DecisionTreeClassifier() 	 0.6586728098108985
DecisionTreeClassifier(splitter='random') 	 0.58461713355611
DecisionTreeClassifier(criterion='entropy') 	 0.6579131719647537
DecisionTreeClass

In [79]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

DecisionTreeClassifier()

In [80]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.686532689438426


In [81]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [82]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.888889,0.450128,0.379913,0.447619,0.54992,0.558473
1,1.0,0.386189,0.244541,0.419048,0.740944,0.545213
2,0.962963,0.43734,0.244541,0.32381,0.529776,0.984089
3,0.851852,0.424552,0.371179,0.485714,0.463509,0.626094
4,0.481481,0.299233,0.174672,0.504762,0.480827,0.653673
...,...,...,...,...,...,...
458,0.925926,0.434783,0.323144,0.414286,0.484538,1.0
459,0.555556,0.304348,0.200873,0.52381,0.417918,0.466985
460,0.407407,0.109974,0.069869,0.742857,0.66973,0.431848
461,0.777778,0.419437,0.200873,0.309524,0.340343,1.0


In [83]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [84]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.2935171264213227
0.24486053167441488
0.23501196795986148
0.23488221449697175


In [85]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=5)
0.2935171264213227


In [96]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25),
    SVR(kernel='linear'),
    SVR(kernel='poly'),
    SVR(kernel='rbf'),
    SVR(kernel='sigmoid'),
    DecisionTreeRegressor(criterion='squared_error'),
    DecisionTreeRegressor(criterion='friedman_mse'),
    DecisionTreeRegressor(criterion='absolute_error')
]

In [97]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[SVR(kernel='sigmoid'), KNeighborsRegressor(), Lasso(), ElasticNet(l1_ratio=0.25)]


In [98]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv", decimal=',')
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,-3.0,-0.3,-0.4
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,-2.0,0.6,-1.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,-7.0,-1.8,-0.3
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,-1.0,1.2,-1.3
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,-2.0,2.5,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,-1.0,5.9,1.5
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,-1.0,5.2,3.1
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,1.0,1.4,1.9
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,-4.0,2.7,1.7


In [99]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = 'water_level'
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['pressure_change', 'temperature', 'pressure', 'visibility', 'wind']


In [100]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

data = data.drop(['precipitation','temperature','visibility','wind','wl_change','temp_change'],axis=1)

In [101]:
data

,water_level,humidity,weather,pressure,fire,pressure_change
0,468.0,31.0,0.0,993.8,4.0,-0.4
1,140.0,22.0,0.0,1004.1,4.0,-1.0
2,464.0,28.0,1.0,1001.2,4.0,-0.3
3,467.0,26.0,0.0,992.3,4.0,-1.3
4,138.0,22.0,0.0,1001.3,4.0,-1.3
...,...,...,...,...,...,...
1434,474.0,95.0,3.0,999.9,4.0,1.5
1435,65.0,93.0,3.0,983.9,4.0,3.1
1436,159.0,83.0,1.0,969.7,4.0,1.9
1437,99.0,91.0,3.0,995.4,4.0,1.7


In [102]:
data.shape
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [103]:
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [104]:
train.head()

,water_level,humidity,weather,pressure,fire,pressure_change
1335,125.0,47.0,0.0,997.3,4.0,-1.5
286,593.0,59.0,1.0,989.6,4.0,-0.8
736,361.0,19.0,0.0,989.2,4.0,-2.5
1334,127.0,58.0,1.0,998.6,4.0,-0.4
1332,172.0,76.0,1.0,1000.4,4.0,-0.6


In [105]:
columns_to_scale = ['fire', 'weather', 'pressure', 'pressure_change', 'humidity']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['fire', 'weather', 'pressure',
                                  'pressure_change', 'humidity'])])

In [106]:
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))
column_names = columns_to_scale + ['water_level']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names
sc_val

,fire,weather,pressure,pressure_change,humidity,water_level
0,0.0,0.125,0.936652,0.526316,0.780220,49.0
1,0.0,0.750,0.746606,0.526316,0.923077,422.0
2,0.0,0.625,0.576169,0.565789,0.956044,285.0
3,0.0,0.125,0.847662,0.421053,0.373626,131.0
4,0.0,0.000,0.773002,0.381579,0.461538,232.0
...,...,...,...,...,...,...
283,0.0,0.000,0.985671,0.500000,0.252747,293.0
284,0.0,0.000,0.079940,0.407895,0.681319,164.0
285,0.0,0.000,0.853695,0.486842,0.461538,509.0
286,0.0,0.125,0.699849,0.605263,0.549451,181.0


In [107]:
r_models = []

r_models.append(ElasticNet(alpha=0.5))
r_models.append(ElasticNet(alpha=0.5, l1_ratio=0.75))

r_models.append(SVR())

r_models.append(Ridge())
r_models

[ElasticNet(alpha=0.5), ElasticNet(alpha=0.5, l1_ratio=0.75), SVR(), Ridge()]

In [108]:
x_labels = column_names[0:-1]
y_labels = ['water_level']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [109]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

ElasticNet(alpha=0.5)
ElasticNet(alpha=0.5, l1_ratio=0.75)
SVR()
Ridge()


In [110]:
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

ElasticNet(alpha=0.5) 	 47175.12253651675
ElasticNet(alpha=0.5, l1_ratio=0.75) 	 46755.00411555744
SVR() 	 52524.78098219462
Ridge() 	 46384.51146320862


In [111]:
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

Ridge()

In [112]:
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

42206.310733745944


In [119]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25),
    SVC(kernel='linear'),
    SVC(kernel='poly'),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid'),
    DecisionTreeClassifier(criterion='gini', splitter='best'),
    DecisionTreeClassifier(criterion='gini', splitter='random'),
    DecisionTreeClassifier(criterion='entropy', splitter='best'),
    DecisionTreeClassifier(criterion='entropy', splitter='random')
]

In [120]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[SVC(kernel='linear'), SVC(kernel='poly'), LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'), KNeighborsClassifier(n_neighbors=10), MultinomialNB(alpha=0.5)]


In [121]:
# Загрузим данные для задачи классификации
newdata = pd.read_csv("datasets/zoo2.csv")
newdata

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [123]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(newdata.columns)
n_x = 8

y_newlabel = 'class_type'
x_newlabels = random.sample(columns[1:-1], n_x)

print(x_newlabels)

['milk', 'domestic', 'toothed', 'legs', 'feathers', 'breathes', 'venomous', 'backbone']


In [124]:
labels = x_newlabels.copy()
labels.append(y_newlabel)
temp_data = newdata[labels]
sec_data = temp_data

In [125]:
sec_data

,milk,domestic,toothed,legs,feathers,breathes,venomous,backbone,class_type
0,0,1,0,4,0,1,0,1,3
1,0,1,1,4,0,1,0,1,3
2,0,1,1,4,0,1,0,1,3
3,0,0,1,4,0,1,0,1,3
4,0,1,1,4,0,1,0,1,3
5,0,0,1,0,0,1,1,1,3
6,0,0,1,0,0,1,0,1,3
7,0,0,1,0,0,1,1,1,3
8,0,0,1,4,0,1,0,1,3
9,0,0,1,4,0,1,0,1,3


In [129]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

sec_data.shape
val_test_size2 = round(0.2*len(sec_data))
print(val_test_size2)

9


In [130]:
random_state = my_seed
train_val2, test2 = train_test_split(sec_data, test_size=val_test_size2, random_state=random_state)
sec_train, val2 = train_test_split(train_val2, test_size=val_test_size2, random_state=random_state)
print(len(train2), len(val2), len(test2))

NameError: name 'train2' is not defined

In [131]:
sec_train.head()

,milk,domestic,toothed,legs,feathers,breathes,venomous,backbone,class_type
8,0,0,1,4,0,1,0,1,3
42,0,0,0,0,0,0,0,0,7
29,0,0,0,6,0,1,0,0,6
11,0,0,1,4,0,1,0,1,3
3,0,0,1,4,0,1,0,1,3


In [132]:
columns_to_scale2 = x_newlabels

ct2 = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale2)], remainder='passthrough')
ct2.fit(sec_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['milk', 'domestic', 'toothed', 'legs',
                                  'feathers', 'breathes', 'venomous',
                                  'backbone'])])

In [133]:
sc_train2 = pd.DataFrame(ct2.transform(sec_train))
sc_test2 = pd.DataFrame(ct2.transform(test2))
sc_val2 = pd.DataFrame(ct2.transform(val2))
column_names2 = columns_to_scale2 + ['class_type']
sc_train2.columns = column_names2
sc_test2.columns = column_names2
sc_val2.columns = column_names2
sc_val2

,milk,domestic,toothed,legs,feathers,breathes,venomous,backbone,class_type
0,0.0,0.0,1.0,0.333333,0.0,1.0,0.0,1.0,5.0
1,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0,6.0
2,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0,5.0
3,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0,6.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,4.0
5,0.0,0.0,1.0,0.000000,0.0,1.0,0.0,1.0,3.0
6,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0,6.0
7,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0,5.0
8,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0,6.0


In [134]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='l1', solver='saga'))


# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier())
c_models.append(DecisionTreeClassifier(splitter='random'))
c_models
[LogisticRegression(penalty='l1', solver='saga'),
 KNeighborsClassifier(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random')]
x_labels2 = column_names2[0:-1]
y_labels2 = ['class_type']

x_train2 = sc_train2[x_labels2]
x_test2 = sc_test2[x_labels2]
x_val2 = sc_val2[x_labels2]

y_train2 = np.ravel(sc_train2[y_labels2])
y_test2 = np.ravel(sc_test2[y_labels2])
y_val2 = np.ravel(sc_val2[y_labels2])
x_train2

,milk,domestic,toothed,legs,feathers,breathes,venomous,backbone
0,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0
4,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0
5,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0
7,0.0,1.0,1.0,0.666667,0.0,1.0,0.0,1.0
8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0
9,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0


In [136]:
# Обучаем модели
for model2 in c_models:
    print(model2)
    model2.fit(x_train2, y_train2)

LogisticRegression(penalty='l1', solver='saga')
KNeighborsClassifier()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [137]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model2 in c_models:
    val_pred2 = model2.predict(x_val2)
    f1 = f1_score(y_val2, val_pred2, average='weighted')
    f1s.append(f1)
    print(model2, '\t', f1)

LogisticRegression(penalty='l1', solver='saga') 	 0.6000000000000001
KNeighborsClassifier() 	 0.6000000000000001
SVC(kernel='sigmoid') 	 0.6000000000000001
DecisionTreeClassifier() 	 0.6000000000000001
DecisionTreeClassifier(splitter='random') 	 0.6000000000000001


In [138]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

LogisticRegression(penalty='l1', solver='saga')

In [139]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred2 = best_c_model.predict(x_test2)
f1 = f1_score(y_test2, test_pred2, average='weighted')
print(f1)

0.7301587301587301


In [151]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=40))

In [152]:
# Выделим используемые параметры
x_labels2 = column_names2[0:-1]
x_data2 = pd.concat([sc_train2[x_labels2], sc_val2[x_labels2], sc_test2[x_labels2]])
x_data2

,milk,domestic,toothed,legs,feathers,breathes,venomous,backbone
0,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0
4,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0
5,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,1.000000,0.0,1.0,0.0,0.0
7,0.0,1.0,1.0,0.666667,0.0,1.0,0.0,1.0
8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0
9,0.0,0.0,1.0,0.666667,0.0,1.0,0.0,1.0


In [153]:
# Произведем кластеризацию
for model2 in k_models:
    model2.fit(x_data2)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (14) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (14) found smaller than n_clusters (40). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
# Оценим качество результата
sils = []
for model2 in k_models:
    cluster_labels = model2.predict(x_data2)
    s = silhouette_score(x_data2, cluster_labels)
    sils.append(s)
    print(s)

0.6545161575655342
0.7982822833012265
0.8372093023255814
0.8372093023255814


In [155]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=20)
0.8372093023255814
